# Prediction of Tanzanian water pumps functional status

This study will focus on identifying the functional status (functional, needs repair or non-functional) of Tanzanian water pumps. The possible explanatory variables will be location, construction year, funder, type of extraction, water quality and quantity, population using it and management organization.

I picked up this challenge from the [DrivenData](https://www.drivendata.org/) competitions list because it shows a direct and pratical application of how statistical analysis can help improve services and products quality. And as an engineer, those goals will be definitely the basis of any data science case I will have to solve. Moreover, as lots of possible explanatory variables are available, this will give me the chance to apply advance tools I learnt during the [Data Analysis and Interpretation online Specialization](https://www.coursera.org/specializations/data-analysis).

Predicting accurately the water pumps functional status will help planning maintenance earlier. That in turn will increase the availability of the water point and thus the quality of life for the people depending on those water supplies.

## Methods

### Sample

The database contains 74,250 records of water points information from the Tanzania Ministry of Water. The records were made between October 2002 and December 2013.

### Measures

The functional status of the water points are categorized in three groups: functional, functional needs repair and non functional. As a hierarchy can be associated to those category, a quantitative variable will be created associating the value 0 as non functional, 1 as functional needs repair and 2 as functional.

The potential predictors will be:

- The amount of water available; missing data are coded as 0, they will be coded as NaN.
- The organization having funded the well; from the various actors, the 9 most funders will be kept and the others will be gathered in an unique category.
- The installer of the well; from the various actors, the 9 most funders will be kept and the others will be gathered in an unique category.
- The GPS coordinates (height, longitude and latitude); missing data are coded as 0, they will be coded as NaN.
- The geographic water basin
- The geographic region
- The population around the well; missing data are coded as 0, they will be coded as NaN.
- Organization of public meeting at the water point; dichotomic variable (True/False)
- The operator of the waterpoint
- The management of the waterpoint
- Is the waterpoint is permitted
- Year the waterpoint was constructed; missing data are coded as 0, they will be coded as NaN.
- The kind of extraction the waterpoint uses
- How payment are handled?
- The quality of the water
- The quantity of the water
- The source of the water
- The type of water point

### Analyses

The distributions of the response and explanatory variables will be evaluated by looking at the frequency tables for categorical variables and by calculating statistical values (mean, standard deviation, minimum and maximum) for quantitative variables.

Bivariate associations will be analyzed by visualizing scatter and box plots. And possible bivariate associations between the explanatory variables and the functional status of the water point will be tested using Pearson correlation (when testing a quantitative variable) or the Analysis of variance, ANOVA, when testing a categorical variable.

The random forest method will be applied to identify the best subset of predictors. The DrivenData competition has split the database in a training set containing 80% of the records and 20% are kept for testing by submission on the website.

`TODO continue`

> This Jupyter notebook will be the basis for the final report for the [Data Analysis and Interpretation Specialization](https://www.coursera.org/specializations/data-analysis)